In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from transformers import ViTFeatureExtractor, ViTForImageClassification
import pandas as pd
import glob
import os

import random
import shutil

In [8]:
# !pip install --upgrade torch torchvision -q

In [9]:
from transformers import AutoFeatureExtractor, AutoModel

# model_ckpt = "nateraw/vit-base-beans"
# extractor = AutoFeatureExtractor.from_pretrained(model_ckpt)
# model = AutoModel.from_pretrained(model_ckpt)
# hidden_dim = model.config.hidden_size

In [10]:
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

# Modify the model's head
# model.classifier = nn.Linear(model.config.hidden_size, 1)


c:\Users\sri.karan\.conda\envs\azureml\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sri.karan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\sri.karan\.conda\envs\azureml\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:31: FutureWarning: The c

In [11]:
# model.classifier = nn.Linear(model.config.hidden_size, 2)

In [12]:
# model_ckpt = "nateraw/vit-base-beans"
# extractor = AutoFeatureExtractor.from_pretrained(model_ckpt)
# model = AutoModel.from_pretrained(model_ckpt)
# hidden_dim = model.config.hidden_size

In [15]:
# import torchvision.transforms as T


# Data transformation chain.
transformation_chain = transforms.Compose(
    [
        # We first resize the input image to 256x256 and then we take center crop.
        transforms.Resize(int((256 / 224) * extractor.size["height"])),
        transforms.CenterCrop(extractor.size["height"]),
        transforms.ToTensor(),
        transforms.Normalize(mean=extractor.image_mean, std=extractor.image_std),
    ]
)

In [16]:
batch_size = 24
device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
import os

In [22]:
# import torchvision.transforms.functional as functional
#http://vision.stanford.edu/aditya86/ImageNetDogs/

In [20]:
class ImagePairDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
#         self.transform=transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
        self.transform = transforms.Compose(
    [
        # We first resize the input image to 256x256 and then we take center crop.
        transforms.Resize(int((256 / 224) * extractor.size["height"])),
        transforms.CenterCrop(extractor.size["height"]),
        transforms.ToTensor(),
        transforms.Normalize(mean=extractor.image_mean, std=extractor.image_std),
    ]
)
        self.image_pairs = self.load_image_pairs()

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        image1_path, image2_path, label = self.image_pairs[idx]
        image1 = Image.open(image1_path).convert("RGB")
        image2 = Image.open(image2_path).convert("RGB")
        
    
        # Convert the tensor to a PIL image
        # image1 = functional.to_pil_image(image1)
        # image2 = functional.to_pil_image(image2)
        
        image1 = self.transform(image1)
        image2 = self.transform(image2)
        # image1 = torch.clamp(image1, 0, 1)
        # image2 = torch.clamp(image2, 0, 1)
        return image1, image2, label

    def load_image_pairs(self):
        image_pairs = []
        # Assume the directory structure is as follows:
        # root_dir
        # ├── similar
        # │   ├── similar_image1.jpg
        # │   ├── similar_image2.jpg
        # │   └── ...
        # └── dissimilar
        #     ├── dissimilar_image1.jpg
        #     ├── dissimilar_image2.jpg
        #     └── ...
        similar_dir = os.path.join(self.root_dir, "similar_all_images")
        dissimilar_dir = os.path.join(self.root_dir, "dissimilar_all_images")

        # Load similar image pairs with label 1
        similar_images = os.listdir(similar_dir)
        for i in range(len(similar_images) // 2):
            image1_path = os.path.join(similar_dir, f"similar_{i}_first.jpg")
            image2_path = os.path.join(similar_dir, f"similar_{i}_second.jpg")
            image_pairs.append((image1_path, image2_path, 1))

        # Load dissimilar image pairs with label 0
        dissimilar_images = os.listdir(dissimilar_dir)
        for i in range(len(dissimilar_images) // 2):
            image1_path = os.path.join(dissimilar_dir, f"dissimilar_{i}_first.jpg")
            image2_path = os.path.join(dissimilar_dir, f"dissimilar_{i}_second.jpg")
            image_pairs.append((image1_path, image2_path, 0))

        return image_pairs


In [23]:
def pick_random_files(folder_path,num_files):
    files=os.listdir(folder_path)
    random.shuffle(files)
    
    picked_files=files[:num_files]
    return picked_files

In [24]:
def copy_files(source_folder,files_list,des):
    for file in files_list:
        source_file=os.path.join(source_folder,file)
        
        des_file=os.path.join(des,file)
        shutil.copy2(source_file,des_file)
        
        print(f"Copied {file} to {des}")
    return 

In [25]:
def move_files(source_folder,des):
    files_list=os.listdir(source_folder)
    for file in files_list:
        source_file=os.path.join(source_folder,file)
        
        des_file=os.path.join(des,file)
        shutil.move(source_file,des_file)
        
        print(f"Copied {file} to {des}")
    return 

In [26]:
def rename_file(file_path,new_name):
    directory=os.path.dirname(file_path)
    new_file_path=os.path.join(directory,new_name)
    
    os.rename(file_path,new_file_path)
    print(f"File renamed to {new_file_path}")
    
    
    return

In [27]:
folder_path=r"C:\Users\sri.karan\Downloads\images1\Images\*"
op_path_similar=r"C:\Users\sri.karan\Downloads\images1\Images\similar_all_images"
tmp=r"C:\Users\sri.karan\Downloads\images1\Images\tmp"

In [28]:
op_path_dissimilar=r"C:\Users\sri.karan\Downloads\images1\Images\dissimilar_all_images"

In [29]:
folders_list=glob.glob(folder_path)
folders_list=list(set(folders_list).difference(set(['C:\\Users\\sri.karan\\Downloads\\images1\\Images\\similar_all_images','C:\\Users\\sri.karan\\Downloads\\images1\\Images\\tmp','C:\\Users\\sri.karan\\Downloads\\images1\\Images\\dissimilar_all_images'])))
folders_list

['C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02116738-African_hunting_dog',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02107574-Greater_Swiss_Mountain_dog',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02089078-black-and-tan_coonhound',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02108000-EntleBucher',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02110063-malamute',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02085620-Chihuahua',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02096585-Boston_bull',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02105641-Old_English_sheepdog',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02097474-Tibetan_terrier',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02106382-Bouvier_des_Flandres',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02107312-miniature_pinscher',
 'C:\\Users\\sri.karan\\Downloads\\images1\\Images\\n02110185-Siberian_husky',
 'C:\\Users\\sri.karan\\Do

In [30]:
l,g=0,0

random.shuffle(folders_list)

In [31]:
for i in glob.glob(folder_path):
    if i in ['C:\\Users\\sri.karan\\Downloads\\images1\\Images\\similar_all_images','C:\\Users\\sri.karan\\Downloads\\images1\\Images\\tmp','C:\\Users\\sri.karan\\Downloads\\images1\\Images\\dissimilar_all_images']:
        continue
    
    file_name=i.split('\\')[-1].split("-")[1]
    picked_files=pick_random_files(i,6)
    
    copy_files(i,picked_files,tmp)
    
    for m in range(3):
        rename_file(os.path.join(tmp,picked_files[m*2]),"similar_"+str(g)+"_first.jpg")
        rename_file(os.path.join(tmp,picked_files[m*2+1]),"similar_"+str(g)+"_second.jpg")
        g+=1
    move_files(tmp,op_path_similar)
    choice_one,choice_two=random.choice(range(len(folders_list))),random.choice(range(len(folders_list)))
    picked_dissimilar_one=pick_random_files(folders_list[choice_one],3)
    picked_dissimilar_two=pick_random_files(folders_list[choice_two],3)
    copy_files(folders_list[choice_one],picked_dissimilar_one,tmp)
    copy_files(folders_list[choice_two],picked_dissimilar_two,tmp)
    picked_files_dissimilar=picked_dissimilar_one+picked_dissimilar_two

    for m in range(3):
        rename_file(os.path.join(tmp,picked_files_dissimilar[m]),"dissimilar_"+str(l)+"_first.jpg")
        rename_file(os.path.join(tmp,picked_files_dissimilar[m+3]),"dissimilar_"+str(l)+"_second.jpg")
        l+=1
    move_files(tmp,op_path_dissimilar)

Copied n02085620_12101.jpg to C:\Users\sri.karan\Downloads\images1\Images\tmp
Copied n02085620_712.jpg to C:\Users\sri.karan\Downloads\images1\Images\tmp
Copied n02085620_2204.jpg to C:\Users\sri.karan\Downloads\images1\Images\tmp
Copied n02085620_2507.jpg to C:\Users\sri.karan\Downloads\images1\Images\tmp
Copied n02085620_4980.jpg to C:\Users\sri.karan\Downloads\images1\Images\tmp
Copied n02085620_13383.jpg to C:\Users\sri.karan\Downloads\images1\Images\tmp
File renamed to C:\Users\sri.karan\Downloads\images1\Images\tmp\similar_0_first.jpg
File renamed to C:\Users\sri.karan\Downloads\images1\Images\tmp\similar_0_second.jpg
File renamed to C:\Users\sri.karan\Downloads\images1\Images\tmp\similar_1_first.jpg
File renamed to C:\Users\sri.karan\Downloads\images1\Images\tmp\similar_1_second.jpg
File renamed to C:\Users\sri.karan\Downloads\images1\Images\tmp\similar_2_first.jpg
File renamed to C:\Users\sri.karan\Downloads\images1\Images\tmp\similar_2_second.jpg
Copied similar_0_first.jpg to 

In [36]:
dataset = ImagePairDataset(r"C:\Users\sri.karan\Downloads\images1\Images")


In [37]:
# dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, shuffle=False)


In [38]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [39]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [40]:
for batch in train_loader:
        image1, image2, labels = batch
        break

In [41]:
image1.shape,image2.shape,labels.shape

(torch.Size([32, 3, 224, 224]),
 torch.Size([32, 3, 224, 224]),
 torch.Size([32]))

In [42]:
model.config.hidden_size

768

In [43]:
op=model.vit(image1)

In [44]:
image1.shape

torch.Size([32, 3, 224, 224])

In [45]:
output=op.last_hidden_state[:,0]

In [46]:
output.shape

torch.Size([32, 768])

In [47]:
output.view(batch_size, -1).shape

torch.Size([32, 768])

In [48]:
torch.cosine_similarity(output.view(batch_size, -1), output.view(batch_size, -1))

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<DivBackward0>)

In [49]:
# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)


In [50]:
# criterion = nn.CrossEntropyLoss()


In [51]:
model=model.to(device)

In [52]:
# model.classifier = nn.Linear(model.config.hidden_size, 2).to(device)


In [53]:
model.num_labels

1000

In [54]:
# model.classifier.

In [56]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_loader:
        image1, image2, labels = batch
        # image1 = torch.clamp(image1, 0, 1)
        # image2 = torch.clamp(image2, 0, 1)
    
        # # # Convert the tensor to a PIL image
        # image1 = functional.to_pil_image(image1)
        # image2 = functional.to_pil_image(image2)
        image1 = image1.to(device)
        image2 = image2.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Extract features from the images
        # inputs = extractor(images=image1, return_tensors="pt").to(device)
        # targets = extractor(images=image2, return_tensors="pt").to(device)
        # inputs = extractor(image1)['pixel_values']
        # targets = extractor(image2)['pixel_values']
        # inputs = image1
        # targets = image2

        # Forward pass
        # outputs = model(pixel_values=inputs, labels=targets)
        outputs1 = model.vit(image1).last_hidden_state[:,0]
        outputs2 = model.vit(image2).last_hidden_state[:,0]
        outputs1 = outputs1.view(batch_size, -1)
        outputs2 = outputs2.view(batch_size, -1)
        similarity_scores = torch.cosine_similarity(outputs1, outputs2)
        # loss = criterion(outputs.logits, labels.float().unsqueeze(1))
        # loss=criterion(outputs1, outputs2, labels)
        loss = criterion(similarity_scores, labels.float())
        # Backward pass and optimization

        loss.backward()
        optimizer.step()
        for name,par in model.named_parameters():
            if par.requires_grad:
                # print(name,par.data.sum())
                pass
        total_loss += loss.item()

    # Print the average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}")

# Evaluation loop



Epoch [1/10], Average Loss: 0.5517
Epoch [2/10], Average Loss: 0.4882
Epoch [3/10], Average Loss: 0.4533
Epoch [4/10], Average Loss: 0.4243
Epoch [5/10], Average Loss: 0.3954
Epoch [6/10], Average Loss: 0.3753
Epoch [7/10], Average Loss: 0.3643
Epoch [8/10], Average Loss: 0.3476
Epoch [9/10], Average Loss: 0.3380
Epoch [10/10], Average Loss: 0.3302


In [ ]:
for name,par in model.named_parameters():
    if par.requires_grad:
        print(name,par.data.sum())

In [ ]:
model.eval()
total_correct = 0
total_samples = 0
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in test_loader:
        image1, image2, labels = batch
        image1 = image1.to(device)
        image2 = image2.to(device)
        labels = labels.to(device)

        # Extract features from the images
        # inputs = extractor(images=image1, return_tensors="pt").to(device)
        # targets = extractor(images=image2, return_tensors="pt").to(device)

        # inputs = image1
        # targets = image2
        # Forward pass
        # outputs = model(pixel_values=inputs, labels=targets)
        outputs1 = model.vit(image1).last_hidden_state[:,0]
        outputs2 = model.vit(image2).last_hidden_state[:,0]
        # predicted_labels = torch.round(torch.sigmoid(outputs.logits))
        similarity_scores = torch.cosine_similarity(outputs1, outputs2)
        predictions = (similarity_scores > 0.5).long()
        # total_correct += (predictions == labels.unsqueeze(1)).sum().item()
        total_samples += labels.size(0)
        correct_predictions += torch.sum(predictions == labels).item()
        total_predictions += len(labels)

# Calculate the accuracy
accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
# model(inputs)

In [ ]:
torch.save(model.state_dict(), "image_similarity_model.pt")

In [ ]:
from transformers import ViTForImageClassification, ViTFeatureExtractor

# Initialize the ViT model and feature extractor
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

# Load the saved model state dict
model.load_state_dict(torch.load("image_similarity_model.pt"))

# Set the model to evaluation mode
model.eval()
